In [1]:
import ee
import geemap
ee.Initialize()

In [2]:
import requests
import pandas as pd
import json
import time

In [12]:
GEOCODE = '2201150 '
Escala = 500;
url = f'http://servicodados.ibge.gov.br/api/v3/malhas/municipios/{GEOCODE}?formato=application/vnd.geo+json'
response = requests.get(url)
geo = response.json();
ee_object = geemap.geojson_to_ee(geo)
geometry = ee_object.geometry()

In [4]:
response

<Response [200]>

In [5]:
Map = geemap.Map(center=[-11.77,-45.76], zoom=8)

Map.addLayer(geometry, {}, "Geometria")

Map.addLayerControl() # This line is not needed for ipyleaflet-based Map.

Map


Map(center=[-11.77, -45.76], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(child…

In [6]:
#Se precisar especificar a geometria manualmente no mapa

#Map.draw_features
#roi = ee.FeatureCollection(Map.draw_features)
#geometry = roi.geometry()
#Map

In [7]:

def clusterization(image):

          
    # Load a pre-computed Landsat composite for input.
    input = image.clip(geometry).select("EVI");
  
    #Make the training dataset.
    training = input.sample(**{
    'region': geometry,
    'scale': Escala,
    'numPixels': 5000
    });

    #Instantiate the clusterer and train it.
    clusterer = ee.Clusterer.wekaKMeans(5).train(training)
    #Cluster the input using the trained clusterer.
    result = input.cluster(clusterer)
 
    #Combine the mean and standard deviation reducers.
    reducers = ee.Reducer.mean()

    #Inicialização Lista de EVI dos clustes
    lista = ee.List([])

    #Iteração sobre os 5 Clusters para calcular a estatística de cada região
    for i in range(5):

        #Cálculo da geometria
        geom = result.select("cluster").eq(i).selfMask().reduceToVectors(**{
        'scale':Escala,
        });



        #Reducer para cálculo do EVI médio de cada cluster
        stats = image.reduceRegion(**{
        'reducer': reducers,
        'geometry':geom.geometry(),
        'scale':Escala,
        'bestEffort':True,
        'maxPixels':10000000000,
        });

        #Armarzena o EVI médio de cada clúster em uma lista
        lista = lista.add(stats.get("EVI"));


    #Cópia da Lista para ordenar os 2 maiores EVIs
    lista2 = lista;


    #Ordenar a lista para pegar os 2 maiores valores de EVI médio nos clusters
    #Agora nossos clustests de interesse estão nas posições 3 e 4 da lista 2
    lista2 = lista2.sort()

    #Pegar as chaves dos clusteres de interesse na lista original
    chaves1 = lista.indexOf(lista2.get(3))
    chaves2 = lista.indexOf(lista2.get(4))


    #Cálculo da Geometria final

    geom1 = result.select("cluster").eq(chaves1).selfMask().reduceToVectors(**{
    'scale':Escala,
    });

    geom2 = result.select("cluster").eq(chaves2).selfMask().reduceToVectors(**{
    'scale':Escala,
    });

    #Geom é a feature collection de nossos clusters de interesse
    geom = geom1.merge(geom2);



    stats = image.reduceRegion(**{
    'reducer': reducers,
    'geometry':geom.geometry(),
    'scale':Escala,
    'bestEffort':True,
    'maxPixels':10000000000,
    });


    return image.set('EVI_Cluster', stats.get("EVI"))
    



In [8]:
import requests
import pandas as pd
import json
import time
#import os

#out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')


stats = list();

In [13]:
#Dividir o processamento das imagens de acordo com o ano para não sobrecarregar o sistema
ano = list(range(2000, 2022))

#string_ints = [str(int) for int in ano]
#stats = list();
    
for i in (ano):
    
    print ("Safra",i-1,"/",i)
    #Filtrar a territorialidade 
    #4 principais meses de safra
    #filter start and end date   
    mcollection = ee.ImageCollection('MODIS/006/MOD13Q1').filterBounds(geometry).filterDate( str(i-1) + '-11-01', str(i-1) +'-12-31').filter(ee.Filter.calendarRange(11, 12,'month')).map(clusterization)
    mcollection2 = ee.ImageCollection('MODIS/006/MOD13Q1').filterBounds(geometry).filterDate( str(i) + '-01-01', str(i) +'-02-28').filter(ee.Filter.calendarRange(1, 2,'month')).map(clusterization)    
    mcollection = mcollection.merge(mcollection2)
    
    #Pegar a propriedade que nos interessa em cada imagem - o EVI_Cluster Calculado 
    a = mcollection.aggregate_array('EVI_Cluster')
    
    
    #10s de soneca pra o GEE não derrubar as requests
    time.sleep(10)
    # Fazer a informação passar do servidor do google para o jupyter
    x = a.getInfo()
    #Salvar cada ano
    stats.append(x)    
    

Safra 1999 / 2000
Safra 2000 / 2001
Safra 2001 / 2002
Safra 2002 / 2003
Safra 2003 / 2004
Safra 2004 / 2005
Safra 2005 / 2006
Safra 2006 / 2007
Safra 2007 / 2008
Safra 2008 / 2009
Safra 2009 / 2010
Safra 2010 / 2011
Safra 2011 / 2012
Safra 2012 / 2013
Safra 2013 / 2014
Safra 2014 / 2015
Safra 2015 / 2016
Safra 2016 / 2017
Safra 2017 / 2018
Safra 2018 / 2019
Safra 2019 / 2020
Safra 2020 / 2021


In [14]:
stats

[[4648.192132335749],
 [4187.9359318363695,
  4067.6739396411094,
  4447.305420761296,
  4456.22914522718,
  4562.811276451284,
  4786.196786369029,
  5156.838287055143],
 [4648.192132335749],
 [4187.9359318363695,
  4067.6739396411094,
  4447.305420761296,
  4456.22914522718,
  4562.811276451284,
  4786.196786369029,
  5156.838287055143],
 [4530.398269828764],
 [4105.572274776766,
  3866.1974138130836,
  4353.674162899956,
  4333.901256681487,
  4491.513012973517,
  4609.159843134976,
  5029.449822962465],
 [3809.2022052231255,
  4165.336913317309,
  4203.195421431562,
  4124.418825936671,
  3840.640309463088,
  4617.349444704459,
  4766.325086493897,
  4217.826660988075],
 [3747.088825478645,
  3582.0407285713436,
  4077.330931548318,
  3779.191400038142,
  4501.927481030977,
  4984.556724224164,
  5110.004016122193,
  4966.372085999531],
 [3749.1709593131764,
  3832.4832280816863,
  4067.416449172959,
  4376.044127296588,
  4709.731717295694,
  4298.6068072742255,
  5515.02783246881

In [125]:
df = pd.DataFrame({'lin':stats})

In [126]:
#Separate comma
#df = df.apply(lambda x:pd.Series(x))

In [127]:
df = pd.DataFrame(stats, columns = ['EVI1', 'EVI2','EVI3','EVI4','EVI5','EVI6','EVI7','EVI8'])

In [128]:
df

,EVI1,EVI2,EVI3,EVI4,EVI5,EVI6,EVI7,EVI8
0,4648.192132,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4187.935932,4067.673940,4447.305421,4456.229145,4562.811276,4786.196786,5156.838287,NaN
2,4648.192132,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4187.935932,4067.673940,4447.305421,4456.229145,4562.811276,4786.196786,5156.838287,NaN
4,4530.398270,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4105.572275,3866.197414,4353.674163,4333.901257,4491.513013,4609.159843,5029.449823,NaN
6,3809.202205,4165.336913,4203.195421,4124.418826,3840.640309,4617.349445,4766.325086,4217.826661
7,3747.088825,3582.040729,4077.330932,3779.191400,4501.927481,4984.556724,5110.004016,4966.372086
8,3749.170959,3832.483228,4067.416449,4376.044127,4709.731717,4298.606807,5515.027832,5434.672804
9,3433.317512,4022.200133,4246.800239,4301.716900,4853.863565,5124.535893,5113.058429,NaN


In [19]:
#Reshape da 1a linha - Imagens do MODIS a partir de Mar/2000
#new_order =  ['EVI6', 'EVI7', 'EVI8','EVI1','EVI2','EVI3','EVI4','EVI5'] # specify new order of the third row
#i = 0 # specify row number
#df.loc[0] = df.loc[0, new_order].values

In [20]:
#Corrigir os Nans
#Features dos EVIS pronta
#Vale a pena talvez pegar os n maiores valores para tentar reduzir o efeito de anos com safra atrasada
df.fillna(0)


,EVI1,EVI2,EVI3,EVI4,EVI5,EVI6,EVI7,EVI8
0,4648.192132,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,4187.935932,4067.673940,4447.305421,4456.229145,4562.811276,4786.196786,5156.838287,0.000000
2,4648.192132,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,4187.935932,4067.673940,4447.305421,4456.229145,4562.811276,4786.196786,5156.838287,0.000000
4,4530.398270,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,4105.572275,3866.197414,4353.674163,4333.901257,4491.513013,4609.159843,5029.449823,0.000000
6,3809.202205,4165.336913,4203.195421,4124.418826,3840.640309,4617.349445,4766.325086,4217.826661
7,3747.088825,3582.040729,4077.330932,3779.191400,4501.927481,4984.556724,5110.004016,4966.372086
8,3749.170959,3832.483228,4067.416449,4376.044127,4709.731717,4298.606807,5515.027832,5434.672804
9,3433.317512,4022.200133,4246.800239,4301.716900,4853.863565,5124.535893,5113.058429,0.000000


In [21]:
url = f'https://apiprevmet3.inmet.gov.br/estacao/proxima/{GEOCODE}'
response = requests.get(url).json()
response

{'error': 'Nenhuma estação próxima encontrada'}

In [22]:
estacao = response['dados']['CD_ESTACAO']

KeyError: 'dados'

In [23]:
new_url = f'https://apitempo.inmet.gov.br/estacao/diaria/1999-11-01/2021-04-30/{estacao}'
json_response = requests.get(new_url).json()
dados_estacao = pd.DataFrame(json_response)
dados_estacao

NameError: name 'estacao' is not defined

In [ ]:
dados_estacao['CHUVA'] = dados_estacao['CHUVA'].astype(float)
dados_estacao['DT_MEDICAO'] = pd.to_datetime(dados_estacao['DT_MEDICAO'])
dados_estacao['ANO'] = dados_estacao['DT_MEDICAO'].dt.year
dados_estacao['MES'] = dados_estacao['DT_MEDICAO'].dt.month
dados_estacao

In [ ]:
b  = dados_estacao.groupby(['ANO', 'MES'])['CHUVA'].sum().fillna(0)

In [ ]:
mask = (dados_estacao['MES'] <= 4) | (dados_estacao['MES'] >= 11)  
b = dados_estacao.loc[mask].groupby(['ANO', 'MES'])['CHUVA'].sum().fillna(0)

In [ ]:
chuva = b.values.reshape(22,6)
df2 = pd.DataFrame(chuva, columns = ['C_NOV', 'C_DEZ','C_JAN','C_FEV','C_MAR','C_ABR'])
df2

In [ ]:
concat_data = pd.concat([df,df2],axis=1).fillna(0)

In [ ]:
concat_data

In [24]:
# Api PAM SIDRA/ https://apisidra.ibge.gov.br/values/t/5457/n6/2919553/v/112/p/last%2020/c782/40124

In [25]:
url3 = f'https://apisidra.ibge.gov.br/values/t/5457/n6/{GEOCODE}/v/112/p/last%2020/c782/40124'
json_response = requests.get(url3).json()
produtividade = pd.DataFrame(json_response)
Y = produtividade['V'].iloc[1:].values.ravel()
Y

array(['2397', '2217', '1152', '2609', '2731', '2760', '2613', '2716',
       '3299', '2841', '2730', '2991', '2937', '1968', '2528', '2831',
       '1184', '3055', '3518', '3155'], dtype=object)

In [26]:
#Produtividade de Soja por ano - PAM/SIDRA
df3 = pd.DataFrame(Y)
df3

,0
0,2397
1,2217
2,1152
3,2609
4,2731
5,2760
6,2613
7,2716
8,3299
9,2841


In [152]:
import sklearn
import numpy as np
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [155]:
#Normalizar o Xtrain
#from sklearn import preprocessing
#from sklearn.preprocessing import StandardScaler
#min_max_scaler = preprocessing.MinMaxScaler()
#X_train_minmax = min_max_scaler.fit_transform(Xtrain)

#Modelagem com os 3 maiores EVIS
df = df.fillna(0)
B = df.apply(lambda x: np.sort(x), axis=1, raw=True)#df = df.assign(max_value=df.values.max(1))
B
#df
#c = ['1st Max','2nd Max','3rd Max']


#xf = (df.apply(lambda x: pd.Series(x.nlargest(3)), axis=1))
#xf.dropna(axis=0)
#xf = xf.fillna(0)
#xf

,EVI1,EVI2,EVI3,EVI4,EVI5,EVI6,EVI7,EVI8
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4648.192132
1,0.000000,4067.673940,4187.935932,4447.305421,4456.229145,4562.811276,4786.196786,5156.838287
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4648.192132
3,0.000000,4067.673940,4187.935932,4447.305421,4456.229145,4562.811276,4786.196786,5156.838287
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4530.398270
5,0.000000,3866.197414,4105.572275,4333.901257,4353.674163,4491.513013,4609.159843,5029.449823
6,3809.202205,3840.640309,4124.418826,4165.336913,4203.195421,4217.826661,4617.349445,4766.325086
7,3582.040729,3747.088825,3779.191400,4077.330932,4501.927481,4966.372086,4984.556724,5110.004016
8,3749.170959,3832.483228,4067.416449,4298.606807,4376.044127,4709.731717,5434.672804,5515.027832
9,0.000000,3433.317512,4022.200133,4246.800239,4301.716900,4853.863565,5113.058429,5124.535893


In [181]:
#X_treino com dados do passado e X_Test com dados do futuro
#Selecionei só 3 medidas do EVI para não ficar muita feature
Xtrain = B.iloc[5:24,4:9].fillna(0)
Xtest =  B.iloc[23:26,4:9].fillna(0)
Ytrain = Y[0:19]
Ytest = Y[19:20]

In [182]:
# Regressão Linear Simples
LR = LinearRegression()
# fitting the training data
LR.fit(Xtrain,Ytrain)
Y_LR =  LR.predict(Xtrain)

In [183]:
#Regressão Polinomial SVR

svr_poly = SVR(kernel='poly', C=1e3, degree=3)
Y_Poly = svr_poly.fit(Xtrain, Ytrain).predict(Xtrain)

In [184]:
#Regressão Ridge
reg = linear_model.Ridge(alpha=.5)
Y_Ridge = reg.fit(Xtrain, Ytrain).predict(Xtrain)

In [185]:
#Regressão Bayseana
regB = linear_model.BayesianRidge()
Y_Bay = regB.fit(Xtrain, Ytrain).predict(Xtrain)

In [186]:
# predicting the accuracy score
score=r2_score(Ytrain,Y_LR)
print("r2 Linear ",score)
print("mean_sqrd_error Linear ==",mean_squared_error(Ytrain,Y_LR))
score=r2_score(Ytrain,Y_Poly)
print("r2 Poly ",score)
print("mean_sqrd_error Polinomial ==",mean_squared_error(Ytrain,Y_Poly))
score=r2_score(Ytrain,Y_Ridge)
print("r2 Ridge ",score)
print("mean_sqrd_error Ridge ==",mean_squared_error(Ytrain,Y_Ridge))
score=r2_score(Ytrain,Y_Bay)
print("r2 Bayseano ",score)
print("mean_sqrd_error Bayseano ==",mean_squared_error(Ytrain,Y_Bay))


r2 Linear  0.32760929550872275
mean_sqrd_error Linear == 236341.22750648813
r2 Poly  0.44099279713718065
mean_sqrd_error Polinomial == 196487.61892020036
r2 Ridge  0.3276092955084946
mean_sqrd_error Ridge == 236341.2275065683
r2 Bayseano  0.0029307729128191795
mean_sqrd_error Bayseano == 350463.74595112604


In [187]:
print("Produtividade em 2019:",Ytest)
print("Produtividade Reg Linear:",LR.predict(Xtest))
print("Produtividade Reg Polinomial:",svr_poly.predict(Xtest))
print("Produtividade Reg Ridge:",reg.predict(Xtest))
print("Produtividade Reg Bayeseana:",regB.predict(Xtest))

Produtividade em 2019: ['3155']
Produtividade Reg Linear: [3075.85274595 2529.80353076 2653.47380879]
Produtividade Reg Polinomial: [3515.7714249  2441.17302015 2791.83025227]
Produtividade Reg Ridge: [3075.85235221 2529.80359521 2653.47388354]
Produtividade Reg Bayeseana: [2586.32186242 2585.99258038 2588.66253437]


In [180]:
#Tentativa de Plotar 1 dimensão de feature x Y
#import matplotlib.pyplot as plt
#import numpy as np

#plt.plot(x_plot,y_plot, 'o')
#plt.plot(x_plot,y_poly)
#plt.plot(x_plot,y_prediction)
#plt.show()

Xtrain

,EVI5,EVI6,EVI7,EVI8
5,4353.674163,4491.513013,4609.159843,5029.449823
6,4203.195421,4217.826661,4617.349445,4766.325086
7,4501.927481,4966.372086,4984.556724,5110.004016
8,4376.044127,4709.731717,5434.672804,5515.027832
9,4301.716900,4853.863565,5113.058429,5124.535893
10,4647.747018,4875.364535,5187.185553,5319.445848
11,4880.682636,4916.055691,5013.556911,5948.722844
12,3915.071428,5453.291956,5735.779512,6626.506648
13,5090.753396,6302.692064,6633.516490,7249.444477
14,5809.421203,6740.656238,7083.480109,7489.166007
